# Cross reflection pattern

This pattern principally similar to the self-reflection pattern, the only different point is reflection is performed by another agent.

In [1]:
from typing import List

from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import AgentMessage
from aap_core.orchestration import ReflectionAgent
from aap_core.types import AgentResponse
from aap_dspy.chain import BaseSignatureAdapter, ChatCausalMultiTurnsChain

import dspy

In [2]:
model_task = dspy.LM(model="ollama_chat/qwen3:4b-thinking-2507-q4_K_M", api_base="http://192.168.55.1:11434", api_key="", cache=False, max_tokens=16384)
model_relfection = dspy.LM(model="ollama_chat/granite3.3:8b", api_base="http://192.168.55.1:11434", api_key="", cache=False, max_tokens=65536)

In [3]:
class QA(dspy.Signature):
    guide: str = dspy.InputField(optional=True)
    context: str = dspy.InputField(optional=True)
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()
    history: dspy.History = dspy.InputField()


class SignatureAdapter(BaseSignatureAdapter[QA]):
    def msg2sig(self, message: AgentMessage) -> List[QA]:
        history = dspy.History(messages=[])
        q, a = None, None
        for response in message.responses:
            if response[0] == "user":
                q = response[1]
            elif response[0] == "tool":
                pass
            elif response[0] == "system":
                pass
            else:
                a = response[1]

            if q is not None and a is not None:
                history.messages.append({"question": q, "answer": a})
                q, a = None, None

        context_str = ""
        if message.context is not None:
            context_str = message.context.get("response", None)
        guide_str = self._prefill_dict.get("guide", "")
        qa = QA(question=message.query, answer="", history=history, guide=guide_str, context=context_str)
        return [qa]

    def sig2msg(self, signatures: List[QA], name: str) -> List[AgentResponse]:
        responses = []
        for signature in signatures:
            responses.append(("user", signature.question))
            responses.append((name, signature.answer))
        return responses

In [ ]:
# Notice that with dspy, we don't have user prompt template, as the signature is doing the job already

predictor = dspy.ChainOfThought(QA)

system_prompt_task = """/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is too long (for example greater than 80 lines), split the logic into multiple smaller functions.
- All functions should have docstring explanation. In the explanation, there should be an simple example to illustrate the function and how to call it.
- The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code
"""
task_adapter = SignatureAdapter.with_prefill({"guide": system_prompt_task})
task_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=task_adapter).with_lm(model_task)
task_chain.final_response_as_context("response")

system_prompt_reflection = """/no_think You are a excellent code reviewer and refactor.
Given a function implementation and it explanation in the context field, your task is to review the code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.
- Make sure the final output only contain code, inline code comment and docstring, nothing else.
- If no modificatoin is made, just return the original implementation"""
# Use different llm for cross-reflection
reflection_adapter = SignatureAdapter.with_prefill({"guide": system_prompt_reflection})
reflection_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=reflection_adapter).with_lm(model_relfection)

def state_callback(state: str):
    print(f"agent state: {state}")

reflection_skill = AgentSkill(
    id='reflection-skill',
    name="reflection skill",
    description="self-reflection skill",
    tags=['reflection']
)
reflection_card = AgentCard(
    name="reflection agent",
    description="self-reflection agent",
    skills=[reflection_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)

reflection_agent = ReflectionAgent(card=reflection_card, chain_task=task_chain, chain_reflection=reflection_chain, state_change_callback=state_callback)

In [5]:
# Take a leetcode as an example. Source: https://leetcode.com/problems/integer-to-roman/description/
query = """Seven different symbols represent Roman numerals with the following values:

Symbol	Value
I	1
V	5
X	10
L	50
C	100
D	500
M	1000
Roman numerals are formed by appending the conversions of decimal place values from highest to lowest. Converting a decimal place value into a Roman numeral has the following rules:

If the value does not start with 4 or 9, select the symbol of the maximal value that can be subtracted from the input, append that symbol to the result, subtract its value, and convert the remainder to a Roman numeral.
If the value starts with 4 or 9 use the subtractive form representing one symbol subtracted from the following symbol, for example, 4 is 1 (I) less than 5 (V): IV and 9 is 1 (I) less than 10 (X): IX. Only the following subtractive forms are used: 4 (IV), 9 (IX), 40 (XL), 90 (XC), 400 (CD) and 900 (CM).
Only powers of 10 (I, X, C, M) can be appended consecutively at most 3 times to represent multiples of 10. You cannot append 5 (V), 50 (L), or 500 (D) multiple times. If you need to append a symbol 4 times use the subtractive form.
Given an integer, convert it to a Roman numeral.

Example 1:
Input: num = 3749
Output: "MMMDCCXLIX"
Explanation:
3000 = MMM as 1000 (M) + 1000 (M) + 1000 (M)
 700 = DCC as 500 (D) + 100 (C) + 100 (C)
  40 = XL as 10 (X) less of 50 (L)
   9 = IX as 1 (I) less of 10 (X)
Note: 49 is not 1 (I) less of 50 (L) because the conversion is based on decimal places

Example 2:
Input: num = 58
Output: "LVIII"
Explanation:
50 = L
 8 = VIII"""

final_message = reflection_agent.execute(AgentMessage(query=query))
for response in final_message.responses:
    name, msg = response
    print("-" * 50)
    print(name)
    print(msg)
    print("-" * 50)

agent state: reflection agent:running




[2025-12-10T10:50:24.609667]

System message:

Your input fields are:
1. `guide` (str): 
2. `context` (str): 
3. `question` (str): 
4. `history` (History):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## guide ## ]]
{guide}

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## history ## ]]
{history}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `guide`, `context`, `question`, `history`, produce the fields `answer`.


User message:

[[ ## guide ## ]]
/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any)

In [6]:
dspy.inspect_history(n=10)





[2025-12-10T10:50:24.609667]

System message:

Your input fields are:
1. `guide` (str): 
2. `context` (str): 
3. `question` (str): 
4. `history` (History):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## guide ## ]]
{guide}

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## history ## ]]
{history}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `guide`, `context`, `question`, `history`, produce the fields `answer`.


User message:

[[ ## guide ## ]]
/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is